In [3]:
import sys
!{sys.executable} -m pip install --upgrade pip
!{sys.executable} -m pip install opencv-python-headless numpy pillow scikit-image scikit-learn tqdm

Defaulting to user installation because normal site-packages is not writeable
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 23.2 MB/s eta 0:00:0000:01
Defaulting to user installation because normal site-packages is not writeable
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.0/54.0 MB 48.1 MB/s  0:00:016m0:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.8/16.8 MB 48.0 MB/s  0:00:006m0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.6/6.6 MB 51.0 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.8/14.8 MB 48.1 MB/s  0:00:006m0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.7/9.7 MB 52.2 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 44.0 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 35.9/35.9 MB 51.2 MB/s  0:00:006m0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13/13 [scikit-image] [scikit-image]-headless]


In [4]:
import os, cv2, numpy as np
from glob import glob

in_dir  = "/workspaces/fruit-detector-pricer/ProyectoFinal/Dataset_Segmentado"   # tus máscaras actuales (blanco=fruta)
out_dir = "/workspaces/fruit-detector-pricer/ProyectoFinal/Morfologia/masks_clean"      # máscaras limpias
os.makedirs(out_dir, exist_ok=True)

# kernel 3x3 cuadrado
kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (3,3))

def clean_mask(mask):
    # asegurar binaria 0/255 y fruta blanca
    mask = (mask > 0).astype(np.uint8) * 255
    # apertura (quita puntitos)
    mask = cv2.morphologyEx(mask, cv2.MORPH_OPEN,  kernel, iterations=1)
    # cierre (tapa huecos pequeños)
    mask = cv2.morphologyEx(mask, cv2.MORPH_CLOSE, kernel, iterations=1)

    # quedarse con la componente más grande
    num, labels, stats, _ = cv2.connectedComponentsWithStats(mask, connectivity=8)
    if num > 1:
        # stats: [label, x, y, w, h, area] -> la 0 es fondo
        largest = 1 + np.argmax(stats[1:, cv2.CC_STAT_AREA])
        mask = np.where(labels == largest, 255, 0).astype("uint8")
    return mask

# recorre subcarpetas (train/test/clase)
paths = [p for p in glob(os.path.join(in_dir, "**", "*.jpg"), recursive=True)] + \
        [p for p in glob(os.path.join(in_dir, "**", "*.png"), recursive=True)]

for src in paths:
    m = cv2.imread(src, cv2.IMREAD_GRAYSCALE)
    if m is None: 
        continue
    cm = clean_mask(m)

    dst = src.replace(in_dir, out_dir)
    os.makedirs(os.path.dirname(dst), exist_ok=True)
    cv2.imwrite(dst, cm)

print("Listo: máscaras limpias en", out_dir)


Listo: máscaras limpias en /workspaces/fruit-detector-pricer/ProyectoFinal/Morfologia/masks_clean
